# Path Optimization (KOMO)



Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [ ]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

C = ry.Config()
C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')

Let's add some objects

In [ ]:
C.addObject(name='item1', parent='sink1', shape=ry.ST.ssBox, pos=[-.1, -.1, .52], size=[.1, .1, .25, .02], color=[1., 0., 0.])
C.addObject('item2', 'sink1', ry.ST.ssBox, [.1, .1, .25, .02], [1., 1., 0.], [.1, .1, .52])
C.addObject('tray', 'stove1', ry.ST.ssBox, [.2, .2, .05, .02], [0., 1., 0.], [.0, .0, .42])
C.view()

compute a collision free path to touch object item1 with pr2L:

In [ ]:
X0 = C.getFrameState()

obj1 = "item1";
arm = "pr2R";

komo = C.komo_path(1.,20, 10., True);

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.distance, [arm, obj1], ry.OT.eq, [1e2]);
komo.addObjective([.9,1.], ry.FS.positionDiff, ["endeffWorkspace", obj1], ry.OT.sos, [1e0]);
komo.addObjective(times=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);

ry.NLP_Solver().setProblem(komo.nlp()).solve()
komo.getReport()

In [ ]:
komo.view(False, "my solution")

In [ ]:
komo.view_play(False, .1)

Best way to read out: grab a configuration into K and analyze it here:

In [ ]:
C.setFrameState( komo.getFrameState(19) )
C.getJointState()
C.view()